# Track Mesoescale Convective Systems (MCS) at consecutive time steps to form tracks with a scheme based on brightness temperature and precipitation

This notebook tracks detected MCS at individual time steps to form tracks. In production, you can use the `scripts/1_detect_and_track_MCS_scheme_Tb_and_P.py` for this step.

ATRACKCS is a Python package for the automated detection and tracking of MCS. It is a potential tool for characterizing their spatio-temporal distribution and evolution. ATRACKCS provides a set of Python functions designed for a workflow analysis  that includes the detection and characterization of  MCS, as well as the integration in tracks, allowing detailed monitoring of the MCS life cycle both in space and time.

ATRACKCS uses brightness temperature (Tb) and precipitation (P) coming from satellite data and can operate with Tb as the only input variable or associating precipitation features. Although the magnitude of Tb does not represent a direct measurement of P, it is an indirect representation of cloud cover height associated with an MCS event. Some methodologies for MCS detection use other satellite spectral bands as a proxy for P. The algorithm parameterization can be adapted to the needs of the MCS detection, as the user is allowed to define the thresholds of Tb and P. The parameterization in this notebook was established from an extensive literature review, which can be consulted below.

## The detection of the MCS (regions) is performed using these steps:

1. At a given time step, the algorithm finds all pixels where `Tb` $\le 225 K$ and defines approximate regions with the convex hull, using a binary structure where the pixels that satisfy the described condition are equal to $1$ and the remaining pixels are equal to $0$.
2. Transform from geographic to plane coordinates and compute an approximate area of the defined regions.
3. Discard all regions whose area is $< 2000 km^2$.
4. Estimate the average, minimum and maximum brightness temperature of those regions.
5. At a given time step, the algorithm finds all pixels where `P` $\ge 2 mm \times {h^{-1}}$ and discard pixels that do not match with this condition.
6. Estimate an approximate area of the precipitation pixels that are contained in each region.
7. Estimate the average and maximum precipitation for each region whose area is $\ge 500 km^2$. The algorithm has the option of discard those regions whose precipitaion area is $< 500 km^2$, but in this case those regions are going to be part of the possible tracks.

## The tracks are performed using these steps:

Specifically, assume we have detected $n$ MSC at time $t$, and $m$ MSC at time $t+1$. There are theoretically $n \times m$ possible associations to link these two groups of MCS. Of course not all of them are meaningful. The rules that are applied in the association process are:

1. **overlapping priority** principle: for any MCS at time t, the MCS with the highest overlap percentage at time t+1 "wins" and is associated with it.
2. The MCS (with lower or no overlap percentages) at time $t+1$ could form a track on their own, and are left to be associated in the next iteration between $t+1$ and $t+2$.
3. No merging or splitting is allowed, any MCS at time $t$ can only be linked to one MCS at time $t+1$, similarly, any MCS at time $t+1$ can only be linked to one MCS at time $t$.
4. All tracks that do not get updated during the $t$ - $t+1$ process terminate. This assumes that no gap in the track is allowed. 
5. In this parameterization, no tracks are discarded based on their total duration. The algorithm has the option for filtering the tracks with a specific minimun duration.

## Input data

**Brightness Temperature:**
NCEP/CPC L3 (Merge IR V1): Spatial and temporal resolution is 4 km and 30 minutes, 
data availability from February 7, 2000 to present. The interest variable of this dataset is `Tb` and the files format must be `netCDF4`.
https://doi.org/10.5067/P4HZB9N27EKU

**Precipitation:**
GPM (IMERG V06B): Spatial and temporal resolution is 10 km and 30 minutes, 
data availability from June 1, 2000 to present. The interest variable of this dataset is `PrecipitationCal` and the files format must be `netCDF4`.
https://doi.org/10.5067/GPM/IMERG/3B-HH/06

We suggest the option `subset/get data` and use `OpenDAP` method for downloading and refining the date range and interest region.

In this case the algorithm is run for 5 days (2019/12/25/ 00 (UTC) - 2019/12/31/ 00 (UTC)) for northern South America. The input data are in the notebooks folder in the repository. The raw data can be downloaded with the links at the top.

## Steps

1. Make sure you have successfully installed the ATRACKCS library.
2. Execute the following code blocks in sequence.

## Results

* `resume_Tb_P_2019_12_24_19_2019_12_31_17.csv`: a csv table listing various attributes for the tracks and MCS associated.
* `map_Tb_P_2019_12_24_19_2019_12_31_10.html` (folium): plot of the geographical locations of the MSC with informations that links to the associaded tracks and features of the MSC.

## Bibliography
* Feng, Z., Leung, L. R., Liu, N., Wang, J., Houze, R. A., Li, J., Hardin, J. C., Chen, D., & Guo, J. (2021). A Global High‐resolution Mesoscale Convective System Database using Satellite‐derived Cloud Tops, Surface Precipitation, and Tracking. Journal of Geophysical Research: Atmospheres. https://doi.org/10.1029/2020jd034202
* Li, J., Feng, Z., Qian, Y., & Leung, L. R. (2020). A high-resolution unified observational data product of mesoscale convective systems and isolated deep convection in the United States for 2004–2017. Earth System Science Data Discussions, October, 1–48. https://doi.org/10.5194/essd-2020-151
* Liu, W., Cook, K. H., & Vizy, E. K. (2019). The role of mesoscale convective systems in the diurnal cycle of rainfall and its seasonality over sub-Saharan Northern Africa. Climate Dynamics, 52(1–2), 729–745. https://doi.org/10.1007/s00382-018-4162-y
* Vizy, E. K., & Cook, K. H. (2018). Mesoscale convective systems and nocturnal rainfall over the West African Sahel: role of the Inter-tropical front. Climate Dynamics, 50(1–2), 587–614. https://doi.org/10.1007/s00382-017-3628-7

### Set paths

First of all we assign the locations to the input (raw data) using `TBDIR` and `PDIR` and output data using `OUTDIR`.

In [3]:
%matplotlib inline
import os

TBDIR=os.path.join('1_input_data', 'tb/')
PDIR=os.path.join('1_input_data', 'p/')

OUTDIR=os.path.join('1_output_data/')

### Parameters used in the MCS detection and tracking process

* `UTM_LOCAL_ZONE`: int, is needed for converting the WGS geodetic coordinate system to plane coordinate system. This is a constant that must be asociated with the interest region. 
* `UTC_LOCAL_HOUR`: int, is needed for converting the raw data hour (UTC) to a local hour (interest region).
* `UTC_LOCAL_SIGN`: str (minus, plus, local), is needed for converting the raw data hour (UTC) to a local hour (interest region).
* `DETECT_SCHEME`: str (Tb, Both), association scheme: Tb or Both (Tb and P). 
* `TB`: int, MCS based on limited maximun threshold cold cloud top.
* `AREA_TB`: int, MCS with a minimun largest area.
* `MIN_P`: int, MCS with a minimun precipitation pixel.
* `AREA_P`: int, MCS with an mimimum area precipitation.
* `DROP_EMPTY_PRECIPITATION`: boolean, if `True` eliminates MCS that do not contain precipitation with the `MIN_P` and `AREA_P` selected. 
* `THRESHOLD_OVERLAPPING_P`: int, percentage overlap limit between MCS to be considered part of the track.
* `LOCATION_FOLIUM`: list (lat, lon), location for center the map_folium.
* `MIN_DURATION`: int, for filtering tracks based on minimun duration.


In [4]:
#32718 is the UTM zone 18S plane coordinate system. 
#It was Used for tracking MCS in South America - Colombia.
UTM_LOCAL_ZONE = 32718 

#UTC-5 is the local hour for Colombia
UTC_LOCAL_HOUR = 5
UTC_LOCAL_SIGN = "minus"

#Scheme of association
DETECT_SCHEME = "Both"

#RANGE Tb 
#Tb [200 k - 240 k]
#RANGE AREA TB
#area_Tb[>999 km]

TB = 225 #[Feng et al.,(2021); Li et al.,(2020)]

AREA_TB = 2000 # [Lui et al., (2019); Vizy & Cook,(2018)] 

#NO RESTRICTIONS AREA_P AND MIN_P

MIN_P = 2 #[Feng et al.,(2021)]  

AREA_P = 500 #[Feng et al.,(2021)]

DROP_EMPTY_PRECIPITATION = False

THRESHOLD_OVERLAPPING_P = 0

LOCATION_FOLIUM = [5, -73.94]

MIN_DURATION = 0

### Importing modules

In [5]:
#--------Import modules-------------------------
from atrackcs.utils import funcs
from atrackcs.detect import detect_mcs
from atrackcs.features import features_Tb, features_P, features_Tracks
from atrackcs.track import track_mcs

### Reading raw Tb and P data.

The reading process is handled with the function `funcs.readNC()`.

Also make sure the output folder exists.

In [6]:
ds = funcs.readNC(pathTb = TBDIR, pathP =PDIR, utc_local_hour = UTC_LOCAL_HOUR, 
                  utc_local_sign = UTC_LOCAL_SIGN)

if not os.path.exists(OUTDIR):
    os.makedirs(OUTDIR)

Complete Tb and P data reading 2019-12-24T19:00 - 2019-12-31T18:00


The detection process is handled with the function `detect_mcs()`.

* `ds` is the `xarray.Dataset` object we just read in.

### Detecting MCS

In [7]:
mcs_l = detect_mcs(ds, detect_scheme = DETECT_SCHEME, Tb = TB, area_Tb = AREA_TB, 
                   utm_local_zone = UTM_LOCAL_ZONE, path_save = None)

MCS detection completed


The MCS Tb features is handled with the function `features_Tb()`.

* `mcs_l` is the `Geopandas.GeoDataFrame` object we just created in the detection process.

In [8]:
mcs_l = features_Tb(mcs_l, ds)

Estimating MCS's brightness temperature attributes: 


100%|██████████████████████████████████████████████████████████████████████████████| 3200/3200 [03:42<00:00, 14.40it/s]


* `mcs_l` is the `geopandas.GeoDataFrame` object we just created in the tracking process.

Each `row` object in `mcs_l` stores a MCS record. We can have a peak into what `msc_l` columns contains so far. 

* `time`: datetime64, hour (UTC-5) for this case.
* `Tb`: float, polygon index after being filtered by the established parameterization.
* `geometry`: geometry, MCS polygon (convex hull). To view the coordinate reference system of the geometry column, access the crs attribute. For this case is `EPSG:32718`.
* `area_Tb`: float, area polygon [$km^2$]
* `centroid_`: geometry, geometric centroid polygon (convex hull). The crs in  this case is `EPSG:32718`.
* `mean_tb`: float, brightness temperature average of the pixels composing the polygon. [$K$]
* `min_tb`: float, brightness temperature min value of the pixels composing the polygon. [$K$]
* `max_tb`: float, brightness temperature max value of the pixels composing the polygon. [$K$]

For this case 3201 MCS were detected with the selected parameterization.

In [9]:
mcs_l

,time,Tb,geometry,area_tb,centroid_,mean_tb,min_tb,max_tb
0,2019-12-24 19:00:00,1.0,"POLYGON ((1385139.077 8889318.481, 1386208.445...",14842.2,POINT (1463228.275 8937057.716),222.4039,212.5513,234.2721
1,2019-12-24 19:00:00,2.0,"POLYGON ((1595866.092 8883549.520, 1585083.176...",3630.7,POINT (1633163.573 8902402.154),219.1902,212.0816,225.6774
2,2019-12-24 19:00:00,4.0,"POLYGON ((2088725.182 8865190.993, 2066639.621...",6145.4,POINT (2126543.231 8906654.165),222.2126,214.0594,234.6743
3,2019-12-24 19:00:00,7.0,"POLYGON ((285900.134 8954798.123, 274850.936 8...",3710.1,POINT (280833.135 9000177.046),222.6613,216.3039,230.5927
4,2019-12-24 19:00:00,8.0,"POLYGON ((681204.449 8966028.400, 637345.833 8...",3526.0,POINT (677052.995 8997699.311),224.8544,218.0482,261.8145
...,...,...,...,...,...,...,...,...
3196,2019-12-31 17:00:00,180.0,"POLYGON ((97079.094 10836238.196, 75168.595 10...",2630.5,POINT (108278.034 10864738.333),218.2107,211.1152,223.0037
3197,2019-12-31 17:00:00,184.0,"POLYGON ((-691865.280 10883122.696, -758907.75...",15303.5,POINT (-748773.702 10970101.458),217.9394,203.1223,247.8695
3198,2019-12-31 17:00:00,185.0,"POLYGON ((-1029700.410 10904530.804, -1051987....",7530.1,POINT (-1010852.082 10956828.001),211.5872,201.9746,225.0643
3199,2019-12-31 17:00:00,186.0,"POLYGON ((174734.077 10879929.160, 163862.710 ...",4026.6,POINT (179987.332 10927972.864),215.6076,205.0069,226.6848


In [10]:
mcs_l.crs

<Projected CRS: EPSG:32718>
Name: WGS 84 / UTM zone 18S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 78°W and 72°W, southern hemisphere between 80°S and equator, onshore and offshore. Argentina. Brazil. Chile. Colombia. Ecuador. Peru.
- bounds: (-78.0, -80.0, -72.0, 0.0)
Coordinate Operation:
- name: UTM zone 18S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

The MCS P features is handled with the function `features_P()`.

In [11]:
mcs_l = features_P(mcs_l, ds, min_precipitation = MIN_P, area_P = AREA_P,
                   drop_empty_precipitation = DROP_EMPTY_PRECIPITATION)

Estimating MCS's precipitation attributes: 


100%|██████████████████████████████████████████████████████████████████████████████| 3200/3200 [03:54<00:00, 13.64it/s]


### Tracking MCS

The tracking process is handled with the function `track_mcs()`. However, these have not been characterized in this phase. The tracks features are handled with the function `features_Tracks()`.

In [12]:
tracks_l = track_mcs(mcs_l, threshold_overlapping_percentage = THRESHOLD_OVERLAPPING_P, utm_local_zone = UTM_LOCAL_ZONE,
                path_save = None)

Estimating trajectories: 


100%|██████████████████████████████████████████████████████████████████████████████| 3200/3200 [05:07<00:00, 10.40it/s]


In [13]:
tracks_l = features_Tracks(tracks_l, initial_time_hour = MIN_DURATION,
                         path_save = OUTDIR)

Estimating distance and direction between geometrics centroids: 


100%|██████████████████████████████████████████████████████████████████████████████| 1207/1207 [00:36<00:00, 33.15it/s]


* `tracks_l` is a `geopandas.GeoDataFrame` object we just created in the tracking process and stores the MCS polygons and the tracks. 
* This `GeoDataFrame` contains exactly the information previously referenced except for the dropping of some features associated to the brightness temperature and the addition of some characteristics associated to the MCS precipitation and some characteristics generated for the tracks.
* The indexing of this object is a `pandas.MultiIndex` since it hierarchically associates an id for each track and an id for each MCS.
* The indexing of this object is encrypted generated using the `uuid` library. To disable encryption for both MCS and tracks, use the parameter `encrypt_index = False` in the `features_Tracks()` function. The index generated when not using encryption is a `int` iteration result  of the tracking process.
* The indexes encryption is useful when processing a long period of time and must iterate for smaller periods of time, for example every 6 months. This is a limitation imposed by the hardware used to run the algorithm.

We can have a peak into what `tracks_l` new columns contains. 


* `belong`: str, encrypted index generated for each track.
* `id_gdf`: str, encrypted index generated for each MCS.
* `geometry`: geometry, polygon. The crs is `EPSG:4326-WGS84`
* `centroid_`: geometry, geometric centroid polygon. The crs is `EPSG:4326`.
* `mean_pp`: float, precipitation average of the pixels composing the polygon [$mm \times {h^{-1}}$].
* `max_pp`: float, precipitation max value of the pixels composing the polygon [$mm \times {h^{-1}}$].
* `intersection_percentage`: float, percentage overlap between MCS [%].
* `distance_c`: float, distance between the overlapping geometric centroids of the MCS [km].
* `direction`: float, direction between the overlapping geometric centroids of the MCS [°].
* `total_duration`: float, total duration of the event or the track. This value is associated to each MCS of the corresponding track [h]. 
* `total_distance`: float, total distance of the event or the track. This value is associated to each MCS of the corresponding track [km]. 
* `mean_velocity`: float, velocity average of the  event or the track. This value is associated to each MCS of the corresponding track [$km \times {h^{-1}}$]. 

In [14]:
tracks_l

time  \
belong        id_gdf                                       
001c2463-06e7 4383-bcad-fc9d0f669f55 2019-12-27 15:00:00   
              452f-89cc-a6336711ff1c 2019-12-27 18:00:00   
              4980-8968-59a8c88925e9 2019-12-27 16:00:00   
              4ef7-9d41-2133e9674c99 2019-12-27 17:00:00   
00321212-27b1 4acb-92f8-fc527943bb68 2019-12-30 06:00:00   
...                                                  ...   
ffcdebf4-efff 42e4-9d7f-a7e3dd6423fb 2019-12-27 19:00:00   
              4478-86af-07c7660c4d93 2019-12-27 21:00:00   
              44d5-ac48-02c19127d00e 2019-12-27 22:00:00   
              47c7-8215-3fd96d947e0b 2019-12-27 20:00:00   
              4dcc-9b3d-70cd4b878ee2 2019-12-27 18:00:00   

                                                                               geometry  \
belong        id_gdf                                                                      
001c2463-06e7 4383-bcad-fc9d0f669f55  POLYGON ((-61.65000 4.55000, -61.95000 4.75000...   
              452f-89cc-a6336711ff1c  POLYGON ((-62.45000 4.65000, -62.35000 4.95000...   
              4980-8968-59a8c88925e9  POLYGON ((-62.05000 4.65000, -62.05000 5.05000...   
              4ef7-9d41-2133e9674c99  POLYGON ((-62.45000 4.65000, -62.45000 4.95000...   
00321212-27b1 4acb-92f8-fc527943bb68  POLYGON ((-73.95000 6.25000, -73.95000 6.35000...   
...                                                                                 ...   
ffcdebf4-efff 42e4-9d7f-a7e3dd6423fb  POLYGON ((-78.75000 1.15000, -78.75000 1.55000...   
              4478-86af-07c7660c4d93  POLYGON ((-78.25000 1.45000, -78.55000 1.75000...   
              44d5-ac48-02c19127d00e  POLYGON ((-78.35000 1.45000, -78.45000 1.55000...   
              47c7-8215-3fd96d947e0b  POLYGON ((-78.35000 1.35000, -78.65000 1.55000...   
              4dcc-9b3d-70cd4b878ee2  POLYGON ((-78.15000 1.75000, -78.35000 1.85000...   

                                      area_tb                  centroid_  \
belong        id_gdf                                                       
001c2463-06e7 4383-bcad-fc9d0f669f55   5835.1  POINT (-61.42484 4.91631)   
              452f-89cc-a6336711ff1c   6781.6  POINT (-61.74337 5.20817)   
              4980-8968-59a8c88925e9  10377.5  POINT (-61.33094 5.14862)   
              4ef7-9d41-2133e9674c99  13484.2  POINT (-61.42739 5.26086)   
00321212-27b1 4acb-92f8-fc527943bb68   2507.0  POINT (-73.68091 6.53948)   
...                                       ...                        ...   
ffcdebf4-efff 42e4-9d7f-a7e3dd6423fb  13810.2  POINT (-78.21297 2.02218)   
              4478-86af-07c7660c4d93  16584.1  POINT (-78.24542 2.30949)   
              44d5-ac48-02c19127d00e   9369.8  POINT (-78.24736 2.20983)   
              47c7-8215-3fd96d947e0b  17444.7  POINT (-78.17741 2.23199)   
              4dcc-9b3d-70cd4b878ee2   5300.6  POINT (-78.12834 2.22013)   

                                       mean_tb  mean_pp   max_pp  \
belong        id_gdf                                               
001c2463-06e7 4383-bcad-fc9d0f669f55  223.1638   3.1587   5.9144   
              452f-89cc-a6336711ff1c  222.7266   3.3044   5.9730   
              4980-8968-59a8c88925e9  217.5831   4.4978   8.4378   
              4ef7-9d41-2133e9674c99  222.3609   3.9170  15.5196   
00321212-27b1 4acb-92f8-fc527943bb68  220.1129      NaN      NaN   
...                                        ...      ...      ...   
ffcdebf4-efff 42e4-9d7f-a7e3dd6423fb  218.7538   5.4267  14.6697   
              4478-86af-07c7660c4d93  220.7396   4.7531  12.2553   
              44d5-ac48-02c19127d00e  224.8508   5.8867  16.7848   
              47c7-8215-3fd96d947e0b  218.1037   5.7096  13.7417   
              4dcc-9b3d-70cd4b878ee2  210.7928   7.8008  23.9949   

                                      intersection_percentage  distance_c  \
belong        id_gdf                                                        
001c2463-06e7 4383-bcad-fc9d0f669f55                     40

In [15]:
tracks_l.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
#Total tracks and MCS detected 

print ("total tracks detected: " + str(len(tracks_l.index.levels[0])) + " and total MCS detected: " + str(len(tracks_l.index.levels[1])))

total tracks detected: 1208 and total MCS detected: 3201


### As the last step, let is reload the tracks results from the local storage and plot the MCS with the help of the folium library.

The read process tracks is handled with the function `funcs.readTRACKS()`.


In [17]:
#-------------------Load results-------------------
tracks_l = funcs.readTRACKS('1_output_data/resume_Tb_P_2019_12_24_19_2019_12_31_17.csv')

#---------Discarding tracks that last less than 10 hours--------
tracks_l = tracks_l.reset_index()
tracks_l = tracks_l[tracks_l.total_duration >= 10].sort_index()
tracks_l = tracks_l.set_index(["belong", "id_gdf"])
tracks_l = tracks_l.sort_index()

print ("total tracks that last less than 10 hours : " + str(len(tracks_l.index.levels[0])) + " and total MCS detected: " + str(len(tracks_l.index.levels[1])))

total tracks that last less than 10 hours : 20 and total MCS detected: 227


The function `funcs.plot_folium()` saves the `.html` result and return the path where was saved.


In [18]:
path_html_folium = funcs.plot_folium(tracks_l, location = LOCATION_FOLIUM, path_save = OUTDIR)

In [19]:
import IPython
iframe = '<iframe src=' + path_html_folium + ' width=1000 height=500></iframe>'
IPython.display.HTML(iframe)